In [1]:
# !pip install pandas_datareader keras seaborn
# !conda install -y -c conda-forge fbprophet
# !pip install pydot graphviz
import boto3
import base64
from botocore.exceptions import ClientError
from IPython.display import display
import pandas_datareader
import pandas as pd
import numpy as np
from keras import Sequential
from keras.layers import Dense, LSTM, InputLayer, Attention
import seaborn as sns
import matplotlib.pyplot as plt
from keras.utils import plot_model

In [2]:
tickers = ['AAPL']
metric = 'low'
pc_metric = f'{metric}_percent_change'
norm_metric = f'{pc_metric}_norm'
def get_secret():
    secret_name = "alpha_vantage"
    region_name = "us-east-2"
    # Create a Secrets Manager client
    session = boto3.session.Session()
    client = session.client(
        service_name='secretsmanager',
        region_name=region_name
    )
    try:
        get_secret_value_response = client.get_secret_value(SecretId=secret_name)
    except ClientError as e:
        display(e)
    else:
        # Decrypts secret using the associated KMS CMK.
        # Depending on whether the secret is a string or binary, one of these fields will be populated.
        if 'SecretString' in get_secret_value_response:
            secret = get_secret_value_response['SecretString']
        else:
            secret = base64.b64decode(get_secret_value_response['SecretBinary'])
    return secret   
def format_dates(daily_stocks_data):
    df = daily_stocks_data.copy() 
    df['date']=df.index
    df.reset_index(inplace=True, drop=True)
    return df
def add_percent_change(daily_stocks_data, metric):
    percents = list()
    for index, row in daily_stocks_data.iterrows():
        old = row[metric]
        try:
            new = daily_stocks_data.iloc[index + 1][metric]
        except Exception as e:
            percents.append(np.nan) ## no next value, so this is undefined
            continue
        percents.append((new-old)/new)
    cp_df = daily_stocks_data.copy()
    cp_df[f'{metric}_percent_change']=percents
    return cp_df
def add_norm(df, label):
    arr = np.array([x*1000 for x in df[label].to_numpy()]).reshape(-1, 1)
#     norm = normalize(arr, norm='l1')
    norm = arr
    new_df = df.copy()
    new_df[f'{label}_norm'] = norm
    return new_df

In [3]:
ALPHA_API_KEY = get_secret()

In [5]:
# daily_stocks_data_raw = pandas_datareader.av.time_series.AVTimeSeriesReader(symbols=tickers, api_key=ALPHA_API_KEY, function='TIME_SERIES_DAILY').read()
daily_stocks_data = format_dates(daily_stocks_data_raw) 
daily_stocks_data = add_percent_change(daily_stocks_data, metric)
daily_stocks_data[daily_stocks_data[pc_metric].isnull()] = 0
daily_stocks_data = add_norm(daily_stocks_data, pc_metric)
display(daily_stocks_data)

,open,high,low,close,volume,date,low_percent_change,low_percent_change_norm
0,22.06,23.5600,22.000,23.00,16873000,2001-03-23,-0.041174,-41.173687
1,23.13,23.7500,21.130,21.78,13115200,2001-03-26,0.035160,35.159817
2,21.94,23.0500,21.900,22.87,9711100,2001-03-27,-0.018605,-18.604651
3,22.08,22.5000,21.500,22.17,10440400,2001-03-28,0.000000,0.000000
4,21.77,23.4500,21.500,22.53,10947600,2001-03-29,-0.007498,-7.497657
...,...,...,...,...,...,...,...,...
5023,120.40,121.1700,119.160,121.03,88105050,2021-03-12,0.010463,10.463378
5024,121.41,124.0000,120.420,123.99,92590555,2021-03-15,0.034439,34.438520
5025,125.70,127.2200,124.715,125.57,115227936,2021-03-16,-0.019446,-19.446443
5026,124.05,125.8599,122.336,124.76,111932636,2021-03-17,-0.016755,-16.755319


In [6]:
def deep_lstm():
    model = Sequential()
    model.add(InputLayer(input_shape=(None,1)))
    model.add(LSTM(1, return_sequences=True))
    model.add(LSTM(1, return_sequences=True))
    model.add(LSTM(1, return_sequences=True))
    model.add(LSTM(1, return_sequences=True))
    model.add(LSTM(1))
    model.compile(loss='mae')
    return model

In [8]:
model = deep_lstm()
model.summary()
# plot_model(model)

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_5 (LSTM)                (None, None, 1)           12        
_________________________________________________________________
lstm_6 (LSTM)                (None, None, 1)           12        
_________________________________________________________________
lstm_7 (LSTM)                (None, None, 1)           12        
_________________________________________________________________
lstm_8 (LSTM)                (None, None, 1)           12        
_________________________________________________________________
lstm_9 (LSTM)                (None, 1)                 12        
Total params: 60
Trainable params: 60
Non-trainable params: 0
_________________________________________________________________
